In [0]:
from pyspark.sql import SparkSession

In [0]:
from pyspark.sql.functions import *

In [0]:
df = spark.read.format('csv').option('inferschema',True).option('header',True)\
            .load('/Volumes/workspace/default/pytutorial/BigMart Sales.csv')
display(df.head(3))

## Scanning Optimization

In [0]:
#df.rdd.getNumPartitions()
df.selectExpr("spark_partition_id()").distinct().count()


#### Changing the default partition size to 128kb


In [0]:
spark.conf.set("spark.sql.files.maxPartitionBytes",131072)

In [0]:
df.selectExpr("spark_partition_id()").distinct().count()


#### Changing the default partition size to 128MB

In [0]:
spark.conf.set("spark.sql.files.maxPartitionBytes",134217728)

### Repartitioning

In [0]:
df_repar = df.repartition(10)

In [0]:
df_repar.selectExpr("spark_partition_id()").distinct().count()


##### Getting partition info

In [0]:
from pyspark.sql.functions import spark_partition_id

df_repar.withColumn("partition_id", spark_partition_id()).display()


#### Data Writing

In [0]:
df.write.format("parquet")\
    .mode("append")\
    .option("path",'/Volumes/workspace/default/pytutorial')\
    .save()

In [0]:
df_new = spark.read.format("parquet")\
              .load('/Volumes/workspace/default/pytutorial')

df_new = df_new.filter(col("Outlet_Location_Type") == 'Tier 1')

In [0]:
#df_new.display()

##### Scanning Optimization

In [0]:
df.write.format("parquet")\
      .mode("append")\
      .partitionBy("Outlet_Location_Type")\
      .option("path","/FileStore/rawdata/parquetWriteOpt")\
      .save()

In [0]:
df_new = spark.read.format("parquet")\
              .load("/FileStore/rawdata/parquetWriteOpt")

df_new = df_new.filter(col("Outlet_Location_Type") == 'Tier 1')

df_new.display()